In [ ]:
import requests
import pandas as pd
import json

# found this api endpoint from inspecting network requests
api_url = "https://api.toastd.in/api/product"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json',
}

all_products = []
page = 0
page_size = 50

print("fetching products...")

while True:
    params = {
        'sort': 'viewCount:desc',  # sort by views = trending
        'page': page,
        'size': page_size
    }
    
    try:
        response = requests.get(api_url, headers=headers, params=params)
        print(f"page {page}: {response.status_code}")
        
        if response.status_code != 200:
            print(f"error on page {page}")
            break
            
        data = response.json()
        
        if 'content' in data and data['content']:
            products = data['content']
            print(f"  got {len(products)} products")
            
            # extract info from each product
            for product in products:
                # function to get price from dict
                def get_price_amount(price_dict):
                    if isinstance(price_dict, dict):
                        return float(price_dict.get('amount', 0))
                    return 0.0
                
                price = get_price_amount(product.get('price'))
                
                # get discount
                discount = 0
                offer = product.get('offer')
                if isinstance(offer, dict):
                    discount_value = offer.get('value', '0')
                    try:
                        discount = float(discount_value)
                    except (ValueError, TypeError):
                        discount = 0
                
                # calculate final price after discount
                final_price = price
                if discount > 0 and price > 0:
                    final_price = price * (1 - discount / 100)
                
                # get images and videos
                main_image = ''
                all_images = []
                all_videos = []
                video_thumbnails = []
                
                files = product.get('files', [])
                if isinstance(files, list) and len(files) > 0:
                    for file in files:
                        if isinstance(file, dict):
                            file_type = file.get('fileType', '')
                            file_url = file.get('url', '')
                            
                            if file_type == 'image' and file_url:
                                all_images.append(file_url)
                                if not main_image:
                                    main_image = file_url
                            elif file_type == 'video' and file_url:
                                all_videos.append(file_url)
                                thumb = file.get('thumbnailUrl', '')
                                if thumb:
                                    video_thumbnails.append(thumb)
                
                # get tags
                tags_list = []
                tags_names = []
                tags = product.get('tags', [])
                if isinstance(tags, list):
                    for tag in tags:
                        if isinstance(tag, dict):
                            tag_name = tag.get('name', '')
                            if tag_name:
                                tags_names.append(tag_name)
                                tags_list.append({
                                    'name': tag_name,
                                    'slug': tag.get('slug', ''),
                                    'color': tag.get('color', ''),
                                    'description': tag.get('description', '')
                                })
                
                # get personas
                personas = product.get('personas')
                personas_str = ''
                if personas is not None:
                    if isinstance(personas, list):
                        personas_str = '|'.join([str(p) for p in personas if p])
                    elif isinstance(personas, str):
                        personas_str = personas
                
                product_info = {
                    'id': product.get('id', ''),
                    'name': product.get('headline', ''),
                    'title': product.get('title', ''),
                    'slug': product.get('slug', ''),
                    'short_description': product.get('shortDescription', ''),
                    'headline_description': product.get('headlineDescription', ''),
                    'meta_title': product.get('metaTitle', ''),
                    'meta_description': product.get('metaDescription', ''),
                    'price': price,
                    'discount': discount,
                    'discount_code': offer.get('code', '') if isinstance(offer, dict) else '',
                    'discount_type': offer.get('type', '') if isinstance(offer, dict) else '',
                    'discount_symbol': offer.get('symbol', '') if isinstance(offer, dict) else '',
                    'final_price': round(final_price, 2),
                    'view_count': product.get('viewCount', 0),
                    'tape_view_count': product.get('tapeViewCount', 0),
                    'vote_count': product.get('upvoteCount', 0),
                    'is_upvoted': product.get('isUpvoted', False),
                    'is_discounted': product.get('isDiscounted', False),
                    'is_active': product.get('isActive', True),
                    'is_discoverable': product.get('isDiscoverable', True),
                    'brand_id': product.get('brandId', ''),
                    'category_id': product.get('mainCategoryId', ''),
                    'tags': '|'.join(tags_names),
                    'tags_json': json.dumps(tags_list) if tags_list else '',
                    'personas': personas_str,
                    'main_image': main_image,
                    'all_images': '|'.join(all_images),
                    'all_videos': '|'.join(all_videos),
                    'video_thumbnails': '|'.join(video_thumbnails),
                    'image_count': len(all_images),
                    'video_count': len(all_videos),
                    'product_url': f"https://www.toastd.in/product/{product.get('slug', '')}",
                    'created_on': product.get('createdOn', ''),
                    'launched_on': product.get('launchedOn', ''),
                    'expiry_date': product.get('expiryDate', ''),
                    'shipping_details': product.get('shippingDetails', ''),
                    'return_policy': product.get('returnPolicy', ''),
                }
                all_products.append(product_info)
            
            if data.get('last', True):
                print(f"last page: {page}")
                break
                
            page += 1
        else:
            print(f"no products on page {page}")
            break
            
    except Exception as e:
        print(f"error on page {page}: {str(e)}")
        break

df = pd.DataFrame(all_products)

print(f"\n{'='*60}")
print(f"total products: {len(df)}")
print(f"{'='*60}")

# some stats
if not df.empty:
    valid_prices = df[df['price'] > 0]
    discounted_products = df[df['is_discounted'] == True]
    
    print("\nsummary:")
    print(f"  products: {len(df)}")
    print(f"  discounted: {len(discounted_products)}")
    print(f"  brands: {df['brand_id'].nunique()}")
    print(f"  categories: {df['category_id'].nunique()}")
    
    if len(valid_prices) > 0:
        print(f"\nprices:")
        print(f"  avg price: ₹{valid_prices['price'].mean():.2f}")
        print(f"  avg final price: ₹{valid_prices['final_price'].mean():.2f}")
        print(f"  range: ₹{valid_prices['price'].min():.0f} - ₹{valid_prices['price'].max():.0f}")
        
        if len(discounted_products) > 0:
            print(f"  avg discount: {discounted_products['discount'].mean():.1f}%")
            print(f"  avg savings: ₹{(discounted_products['price'] - discounted_products['final_price']).mean():.2f}")
    
    print(f"\nengagement:")
    print(f"  total views: {df['view_count'].sum():,}")
    print(f"  total video views: {df['tape_view_count'].sum():,}")
    print(f"  total votes: {df['vote_count'].sum():,}")
    print(f"  avg views: {df['view_count'].mean():.0f}")
    print(f"  avg votes: {df['vote_count'].mean():.1f}")
    
    products_with_tags = df[df['tags'] != ''].shape[0]
    products_with_videos = df[df['video_count'] > 0].shape[0]
    products_with_descriptions = df[df['meta_description'] != ''].shape[0]
    products_active = df[df['is_active'] == True].shape[0]
    
    print(f"\ncontent:")
    print(f"  with tags: {products_with_tags} ({products_with_tags/len(df)*100:.1f}%)")
    print(f"  with videos: {products_with_videos} ({products_with_videos/len(df)*100:.1f}%)")
    print(f"  with descriptions: {products_with_descriptions} ({products_with_descriptions/len(df)*100:.1f}%)")
    print(f"  active: {products_active} ({products_active/len(df)*100:.1f}%)")
    print(f"  avg images: {df['image_count'].mean():.1f}")
    print(f"  avg videos: {df['video_count'].mean():.1f}")
    
    print("\n" + "="*60)
    print("top 10 trending:")
    print("="*60)
    top_10 = df.nlargest(10, 'view_count')[['name', 'view_count', 'vote_count', 'price', 'final_price', 'tags']]
    top_10_display = top_10.copy()
    top_10_display['tags'] = top_10_display['tags'].apply(lambda x: (x[:40] + '...') if len(str(x)) > 40 else x)
    print(top_10_display.to_string(index=False))
    
    print("\n" + "="*60)
    print("sample (first 5):")
    print("="*60)
    sample = df.head()[['name', 'price', 'final_price', 'image_count', 'video_count', 'tags']]
    sample_display = sample.copy()
    sample_display['tags'] = sample_display['tags'].apply(lambda x: (x[:30] + '...') if len(str(x)) > 30 else x)
    print(sample_display.to_string(index=True))
else:
    print("no products scraped!")


Fetching products from Toastd API...
Page 0: Status Code 200
  Found 50 products on page 0
Page 1: Status Code 200
  Found 50 products on page 1
Page 2: Status Code 200
  Found 50 products on page 2
Page 3: Status Code 200
  Found 50 products on page 3
Page 4: Status Code 200
  Found 50 products on page 4
Page 5: Status Code 200
  Found 50 products on page 5
Page 6: Status Code 200
  Found 50 products on page 6
Page 7: Status Code 200
  Found 50 products on page 7
Page 8: Status Code 200
  Found 50 products on page 8
Page 9: Status Code 200
  Found 16 products on page 9
Reached last page at page 9

Total products scraped: 466

Product Catalog Summary:
  Total Products: 466
  Products with Discounts: 347
  Unique Brands: 55 (IDs only)
  Unique Categories: 16 (IDs only)

Pricing:
  Avg Original Price: ₹2712.86
  Avg Final Price: ₹2347.86
  Price Range: ₹40 - ₹25000
  Avg Discount: 19.0%
  Avg Savings: ₹485.99

Engagement:
  Total Views: 1,380,800
  Total Video Views: 0
  Total Votes: 1,7

In [ ]:
# save to csv
if not df.empty:
    output_file = 'toastd_products.csv'
    df.to_csv(output_file, index=False)
    print(f"\nsaved to {output_file}")
    
    # category stats
    df_with_category = df[df['category_id'] != '']
    
    if len(df_with_category) > 0:
        print("\n" + "="*60)
        print("by category:")
        print("="*60)
        category_stats = df_with_category.groupby('category_id').agg({
            'id': 'count',
            'view_count': 'sum',
            'vote_count': 'sum',
            'final_price': 'mean'
        }).rename(columns={
            'id': 'product_count',
            'view_count': 'total_views',
            'vote_count': 'total_votes',
            'final_price': 'avg_price'
        }).sort_values('total_views', ascending=False).head(10)
        
        print(category_stats.to_string())
    
    df_with_brand = df[df['brand_id'] != '']
    
    if len(df_with_brand) > 0:
        print("\n" + "="*60)
        print("top brands:")
        print("="*60)
        brand_counts = df_with_brand['brand_id'].value_counts().head(10)
        print(brand_counts.to_string())
    
    print("\n" + "="*60)
    print("price distribution:")
    print("="*60)
    price_ranges = pd.cut(df[df['price'] > 0]['price'], 
                          bins=[0, 500, 1000, 2000, 5000, 10000, float('inf')],
                          labels=['<₹500', '₹500-1000', '₹1000-2000', '₹2000-5000', '₹5000-10000', '>₹10000'])
    price_dist = price_ranges.value_counts().sort_index()
    print(price_dist.to_string())
    
    print("\n" + "="*60)
    print("discounts:")
    print("="*60)
    discount_dist = df[df['is_discounted'] == True]['discount'].value_counts().sort_index()
    print(discount_dist.to_string())
    
    print("\n" + "="*60)
    print("discount codes:")
    print("="*60)
    code_dist = df[df['discount_code'] != '']['discount_code'].value_counts().head(10)
    print(code_dist.to_string())
    
    print("\n" + "="*60)
    print("common tags:")
    print("="*60)
    all_tags = []
    for tags_str in df[df['tags'] != '']['tags']:
        all_tags.extend(tags_str.split('|'))
    
    if all_tags:
        tag_counts = pd.Series(all_tags).value_counts().head(15)
        print(tag_counts.to_string())
    else:
        print("no tags found")
    
    print("\n" + "="*60)
    print("media:")
    print("="*60)
    print(f"products with 5+ images: {len(df[df['image_count'] >= 5])}")
    print(f"products with videos: {len(df[df['video_count'] > 0])}")
    print(f"max images: {df['image_count'].max()}")
    print(f"max videos: {df['video_count'].max()}")
else:
    print("no data!")



✓ Data saved to toastd_products.csv

Category-wise Analysis (by Category ID):
                                      product_count  total_views  total_votes    avg_price
category_id                                                                               
0181774c-41c6-439d-8d53-75c3848e2a54             96       435682          462  2031.344792
36b291ec-3666-4da1-9f51-82fa2df373a5             53       211752          352   826.907547
eb232420-804d-4165-9cbe-8266ababcbee            128       145787          175  1653.861719
ba4ed8a0-5dac-4390-bc61-9446e374ab3b             30       128333          303  1912.008333
2744b95e-36ee-4abb-b896-225a7d22d728             33       128160           46  3119.145455
3142d0c8-1de1-408f-9ed9-45087b116967             11        80322           71  1244.672727
c89321cf-e6f1-4f98-a0d9-beb4e54ed535             32        75902           18  1577.875000
8045bd2c-d72c-42c8-9152-6cc6a8c62075             17        73856          200  1125.702941
a7d02d6f-c1

In [ ]:
# export simpler version
if not df.empty:
    simple_export = df[[
        'name', 
        'price', 
        'discount', 
        'final_price', 
        'view_count', 
        'vote_count',
        'discount_code',
        'product_url'
    ]].copy()
    
    simple_export_file = 'toastd_products_simple.csv'
    simple_export.to_csv(simple_export_file, index=False)
    print(f"\nsaved simple version: {simple_export_file}")
    
    print("\n" + "="*60)
    print("most voted:")
    print("="*60)
    top_voted = df.nlargest(10, 'vote_count')[['name', 'vote_count', 'view_count', 'final_price']]
    print(top_voted.to_string(index=False))
    
    print("\n" + "="*60)
    print("best deals:")
    print("="*60)
    best_deals = df[df['discount'] > 0].nlargest(10, 'discount')[['name', 'price', 'final_price', 'discount', 'discount_code']]
    print(best_deals.to_string(index=False))
    
    print("\n" + "="*60)
    print("richest content:")
    print("="*60)
    # score based on tags, description, videos, images
    df['content_score'] = (
        (df['tags'] != '').astype(int) * 3 +
        (df['meta_description'] != '').astype(int) * 2 +
        (df['video_count'] > 0).astype(int) * 2 +
        (df['image_count'] >= 5).astype(int)
    )
    
    richest_content = df.nlargest(10, 'content_score')[['name', 'tags', 'image_count', 'video_count', 'content_score']]
    richest_display = richest_content.copy()
    richest_display['tags'] = richest_display['tags'].apply(lambda x: (str(x)[:40] + '...') if len(str(x)) > 40 else x)
    print(richest_display.to_string(index=False))
    
    print("\n" + "="*60)
    print("products with tags:")
    print("="*60)
    products_with_tags = df[df['tags'] != ''].head(10)[['name', 'tags', 'view_count']]
    tags_display = products_with_tags.copy()
    tags_display['tags'] = tags_display['tags'].apply(lambda x: (str(x)[:50] + '...') if len(str(x)) > 50 else x)
    print(tags_display.to_string(index=False))
else:
    print("no data!")



✓ Simplified export saved to toastd_products_simple.csv

Top 10 Most Voted Products:
                                 name  vote_count  view_count  final_price
             Minimalist Barefoot Shoe          96       25337      4319.10
               PureFlow Nasal Strips           95       11035       629.10
       3D Printed portable table lamp          78        1188      5099.15
        DIY Light Bottle Painting Kit          58       26626       764.15
Kitsters X Chefling – Sushi Kit for 2          52       30341      1104.15
                 Dubai Bling Earrings          39        2289       854.10
  2 Air Fiber Glass Paddles & 2 Balls          37        4062      4675.00
  MAGMATE Classic - Fun Magnetic Game          34        2379       479.20
                        HABIT TRACKER          32       21675       269.10
     DIY Solar Printing Cyanotype Kit          32        2072      1487.50

Top 10 Best Deals (Highest Discount %):
                                      name  pric

In [ ]:
# export different datasets for recommendations
if not df.empty:
    print("="*60)
    print("creating specialized exports")
    print("="*60)
    
    # text for search
    text_corpus = df[df['meta_description'] != ''][['id', 'name', 'meta_description', 'headline_description', 'tags', 'short_description']].copy()
    text_corpus.to_csv('toastd_text_corpus.csv', index=False)
    print(f"\ntext corpus: toastd_text_corpus.csv ({len(text_corpus)} products)")
    
    # tags for content filtering
    tags_data = df[df['tags'] != ''][['id', 'name', 'tags', 'tags_json', 'category_id', 'brand_id']].copy()
    tags_data.to_csv('toastd_tags.csv', index=False)
    print(f"tags: toastd_tags.csv ({len(tags_data)} products)")
    
    # media files
    media_data = df[df['image_count'] > 0][['id', 'name', 'main_image', 'all_images', 'all_videos', 'video_thumbnails', 'image_count', 'video_count']].copy()
    media_data.to_csv('toastd_media.csv', index=False)
    print(f"media: toastd_media.csv ({len(media_data)} products)")
    
    # engagement data
    engagement_data = df[['id', 'name', 'view_count', 'tape_view_count', 'vote_count', 'is_upvoted', 'created_on', 'launched_on']].copy()
    engagement_data.to_csv('toastd_engagement.csv', index=False)
    print(f"engagement: toastd_engagement.csv ({len(engagement_data)} products)")
    
    # features for ML
    features_data = df[['id', 'name', 'price', 'final_price', 'discount', 'is_discounted', 
                         'brand_id', 'category_id', 'tags', 'image_count', 'video_count',
                         'view_count', 'vote_count', 'is_active', 'is_discoverable']].copy()
    features_data.to_csv('toastd_features.csv', index=False)
    print(f"features: toastd_features.csv ({len(features_data)} products)")
    
    print(f"\n{'-'*60}")
    print("summary:")
    print(f"{'-'*60}")
    print(f"total: {len(df)}")
    print(f"with descriptions: {len(text_corpus)} ({len(text_corpus)/len(df)*100:.1f}%)")
    print(f"with tags: {len(tags_data)} ({len(tags_data)/len(df)*100:.1f}%)")
    print(f"with media: {len(media_data)} ({len(media_data)/len(df)*100:.1f}%)")
    print(f"active: {len(df[(df['is_active']==True) & (df['is_discoverable']==True)])}")
    
    unique_tags = set()
    for tags_str in df[df['tags'] != '']['tags']:
        unique_tags.update(tags_str.split('|'))
    print(f"\nunique tags: {len(unique_tags)}")
    print(f"avg tags per product: {df[df['tags'] != '']['tags'].apply(lambda x: len(x.split('|'))).mean():.1f}")
    
    print(f"\n{'='*60}")
    print("all done!")
    print(f"{'='*60}")


Creating Specialized Exports for Recommendation Systems

✓ Text corpus saved: toastd_text_corpus.csv (466 products with descriptions)
✓ Tags dataset saved: toastd_tags.csv (389 products with tags)
✓ Media assets saved: toastd_media.csv (466 products with media)
✓ Engagement metrics saved: toastd_engagement.csv (466 products)
✓ Features dataset saved: toastd_features.csv (466 products)

------------------------------------------------------------
Dataset Coverage Summary:
------------------------------------------------------------
Total products scraped: 466
Products with meta descriptions: 466 (100.0%)
Products with tags: 389 (83.5%)
Products with media: 466 (100.0%)
Active & discoverable products: 466

Total unique tags: 77
Avg tags per product: 1.9

✅ All exports complete! Ready for recommendation system.


In [17]:
# Analyze all unique columns across generated CSV files
import glob
import os

print("="*70)
print("COLUMN ANALYSIS - All Generated CSV Files")
print("="*70)

# Define all CSV files generated
csv_files = [
    'toastd_products.csv',
    'toastd_products_simple.csv',
    'toastd_text_corpus.csv',
    'toastd_tags.csv',
    'toastd_media.csv',
    'toastd_engagement.csv',
    'toastd_features.csv'
]

# Dictionary to store columns per file
file_columns = {}
all_columns = set()

print("\n📁 Analyzing CSV files...\n")

for csv_file in csv_files:
    if os.path.exists(csv_file):
        try:
            # Read just the first row to get column names
            temp_df = pd.read_csv(csv_file, nrows=0)
            columns = list(temp_df.columns)
            file_columns[csv_file] = columns
            all_columns.update(columns)
            
            print(f"✓ {csv_file:<35} | {len(columns):>3} columns")
        except Exception as e:
            print(f"✗ {csv_file:<35} | Error: {str(e)}")
    else:
        print(f"✗ {csv_file:<35} | File not found")

print("\n" + "="*70)
print(f"SUMMARY: {len(all_columns)} Unique Columns Across All Files")
print("="*70)

# Print all unique columns sorted alphabetically
print("\n📋 All Unique Column Names (Alphabetically):\n")
for i, col in enumerate(sorted(all_columns), 1):
    print(f"{i:>3}. {col}")

# Show which columns are in which files
print("\n" + "="*70)
print("COLUMN DISTRIBUTION ACROSS FILES")
print("="*70)

# Create a reverse mapping: column -> list of files
column_to_files = {}
for csv_file, columns in file_columns.items():
    for col in columns:
        if col not in column_to_files:
            column_to_files[col] = []
        column_to_files[col].append(csv_file.replace('toastd_', '').replace('.csv', ''))

# Print columns by frequency (how many files they appear in)
print("\n📊 Columns by Frequency:\n")
from collections import Counter

column_freq = {col: len(files) for col, files in column_to_files.items()}
sorted_by_freq = sorted(column_freq.items(), key=lambda x: (-x[1], x[0]))

current_freq = None
for col, freq in sorted_by_freq:
    if freq != current_freq:
        current_freq = freq
        print(f"\n{'─'*70}")
        print(f"Appears in {freq} file(s):")
        print(f"{'─'*70}")
    
    files_str = ', '.join(column_to_files[col])
    print(f"  • {col:<30} → [{files_str}]")

# Show file-specific details
print("\n" + "="*70)
print("DETAILED BREAKDOWN BY FILE")
print("="*70)

for csv_file, columns in file_columns.items():
    print(f"\n📄 {csv_file}")
    print(f"   Columns ({len(columns)}):")
    for col in columns:
        print(f"      • {col}")

print("\n" + "="*70)
print("✅ Analysis Complete!")
print("="*70)


COLUMN ANALYSIS - All Generated CSV Files

📁 Analyzing CSV files...

✓ toastd_products.csv                 |  38 columns
✓ toastd_products_simple.csv          |   8 columns
✓ toastd_text_corpus.csv              |   6 columns
✓ toastd_tags.csv                     |   6 columns
✓ toastd_media.csv                    |   8 columns
✓ toastd_engagement.csv               |   8 columns
✓ toastd_features.csv                 |  15 columns

SUMMARY: 38 Unique Columns Across All Files

📋 All Unique Column Names (Alphabetically):

  1. all_images
  2. all_videos
  3. brand_id
  4. category_id
  5. created_on
  6. discount
  7. discount_code
  8. discount_symbol
  9. discount_type
 10. expiry_date
 11. final_price
 12. headline_description
 13. id
 14. image_count
 15. is_active
 16. is_discounted
 17. is_discoverable
 18. is_upvoted
 19. launched_on
 20. main_image
 21. meta_description
 22. meta_title
 23. name
 24. personas
 25. price
 26. product_url
 27. return_policy
 28. shipping_details
 29.

In [30]:
# Print first product row as dictionary for easy inspection
print("="*70)
print("FIRST PRODUCT - Complete Data Structure")
print("="*70)

if not df.empty:
    # Get first row as dictionary
    first_product = df.iloc[0].to_dict()
    
    print(f"\nProduct: {first_product.get('name', 'N/A')}")
    print(f"ID: {first_product.get('id', 'N/A')}")
    print("\n" + "-"*70)
    print("ALL FIELDS:")
    print("-"*70)
    
    # Print each field nicely formatted
    for key, value in first_product.items():
        # Truncate long values for readability
        value_str = str(value)
        if len(value_str) > 100:
            value_str = value_str[:100] + "..."
        
        print(f"\n{key}:")
        print(f"  {value_str}")
    
    print("\n" + "="*70)
    print("RAW DICTIONARY FORMAT:")
    print("="*70)
    print("\nimport json")
    print("first_product =", json.dumps(first_product, indent=2, default=str))
    
else:
    print("No data available!")


FIRST PRODUCT - Complete Data Structure

Product: India's Most Effective Portable Air Purifier (Blush Pink)
ID: 66d727eb-7e35-488b-8c65-0cadbd554c0c

----------------------------------------------------------------------
ALL FIELDS:
----------------------------------------------------------------------

id:
  66d727eb-7e35-488b-8c65-0cadbd554c0c

name:
  India's Most Effective Portable Air Purifier (Blush Pink)

title:
  Your personal bubble of fresh air.

slug:
  atovio-pebble-portable-air-purifier-blush-pink

short_description:
  portable personal Wearable air purifier blush pink Lightweight wearable Silent compact

headline_description:
  This isn't just a gadget - it's your personal air guardian.

meta_title:
  Buy Atovio Pebble – India’s Most Effective Portable Air Purifier

meta_description:
  Buy Atovio Pebble Air Purifier and eliminate 90% of airborne pollutants. Personal, portable, filter-...

price:
  3199.0

discount:
  20.0

discount_code:
  TOASTD20

discount_type:
  PERCE

In [22]:
prod = pd.read_csv('toastd_products.csv')

In [31]:
first_product = prod.iloc[0].to_dict()

In [32]:
first_product

{'id': '66d727eb-7e35-488b-8c65-0cadbd554c0c',
 'name': "India's Most Effective Portable Air Purifier (Blush Pink)",
 'title': 'Your personal bubble of fresh air.',
 'slug': 'atovio-pebble-portable-air-purifier-blush-pink',
 'short_description': 'portable personal Wearable air purifier blush pink Lightweight wearable Silent compact',
 'headline_description': "This isn't just a gadget - it's your personal air guardian.",
 'meta_title': 'Buy Atovio Pebble – India’s Most Effective Portable Air Purifier',
 'meta_description': 'Buy Atovio Pebble Air Purifier and eliminate 90% of airborne pollutants. Personal, portable, filter-free, maintenance-free. Turbo mode & IIT Kanpur tested technology.',
 'price': 3199.0,
 'discount': 20,
 'discount_code': 'TOASTD20',
 'discount_type': 'PERCENT',
 'discount_symbol': '%',
 'final_price': 2559.2,
 'view_count': 47753,
 'tape_view_count': 0,
 'vote_count': 4,
 'is_upvoted': False,
 'is_discounted': True,
 'is_active': True,
 'is_discoverable': True,
 'br

In [37]:
# convert to json with only the columns we need
import json
import numpy as np

print("="*70)
print("converting to json")
print("="*70)

columns_to_keep = [
    'id', 'name', 'title', 'slug', 'short_description', 
    'headline_description', 'meta_title', 'meta_description', 
    'price', 'tags', 'main_image', 'view_count', 'vote_count', 'product_url'
]

print(f"\nexporting {len(columns_to_keep)} columns:")
for col in columns_to_keep:
    print(f"  - {col}")

prod_filtered = prod[columns_to_keep].copy()

# replace NaN with empty strings
print(f"\ncleaning data...")
nan_count_before = prod_filtered['tags'].isna().sum()
prod_filtered['tags'] = prod_filtered['tags'].fillna('')
print(f"  replaced {nan_count_before} NaN values in tags")

for col in ['short_description', 'headline_description', 'meta_title', 'meta_description']:
    nan_count = prod_filtered[col].isna().sum()
    if nan_count > 0:
        prod_filtered[col] = prod_filtered[col].fillna('')
        print(f"  replaced {nan_count} NaN values in {col}")

products_list = prod_filtered.to_dict(orient='records')

output_json_file = 'products.json'
with open(output_json_file, 'w', encoding='utf-8') as f:
    json.dump(products_list, f, indent=2, ensure_ascii=False)

print(f"\nsaved {len(products_list)} products to {output_json_file}")
print(f"file size: {os.path.getsize(output_json_file) / (1024*1024):.2f} MB")

print(f"\n{'-'*70}")
print("first product:")
print(f"{'-'*70}\n")
print(json.dumps(products_list[0], indent=2, ensure_ascii=False))

print(f"\n{'='*70}")
print("done!")
print(f"{'='*70}")


converting to json

exporting 14 columns:
  - id
  - name
  - title
  - slug
  - short_description
  - headline_description
  - meta_title
  - meta_description
  - price
  - tags
  - main_image
  - view_count
  - vote_count
  - product_url

cleaning data...
  replaced 77 NaN values in tags

saved 466 products to products.json
file size: 0.52 MB

----------------------------------------------------------------------
first product:
----------------------------------------------------------------------

{
  "id": "66d727eb-7e35-488b-8c65-0cadbd554c0c",
  "name": "India's Most Effective Portable Air Purifier (Blush Pink)",
  "title": "Your personal bubble of fresh air.",
  "slug": "atovio-pebble-portable-air-purifier-blush-pink",
  "short_description": "portable personal Wearable air purifier blush pink Lightweight wearable Silent compact",
  "headline_description": "This isn't just a gadget - it's your personal air guardian.",
  "meta_title": "Buy Atovio Pebble – India’s Most Effective Po